In [1]:
import jsonlines
import random

In [2]:
def read_jsonl(file_paths):
    content_id = 0
    preprocessed_data = []
    questions = []
    data = []
    question_id = 0
    for file_path in file_paths:
        with open(file_path, "r") as f:
            for item in jsonlines.Reader(f):
                paper_questions = []
                for section in item["sections"]:
                    start = section[0]
                    end = section[1]
                    content = " ".join(item["words"][start:end])
                    preprocessed_data.append({"id": "scirex-{:0>4d}".format(content_id), "content": content})
                    content_id += 1
                for relation in item["n_ary_relations"]:
                    Material = relation["Material"]
                    Method = relation["Method"]
                    Metric = relation["Metric"]
                    Task = relation["Task"]
                    score = relation["score"]
                    data.append({"Material": Material, "Method": Method, "Metric": Metric, "Task": Task, "score": score})
                    question = "What is the corresponding {} score of the {} method on {} dataset for {} task?".format(Metric, Method, Material, Task)
                    answer = score
                    paper_questions.append({"qid": "", "question": question, "answer": answer})
                random_index = random.randint(0, len(paper_questions) - 1)
                questions.append(paper_questions[random_index])
    return questions

In [3]:
file_paths = ["/<YOUR_OWN_PATH>/ToolQA/data/raw_data/scirex/train.jsonl", 
              "/<YOUR_OWN_PATH>/ToolQA/data/raw_data/scirex/dev.jsonl", 
              "/<YOUR_OWN_PATH>/ToolQA/data/raw_data/scirex/test.jsonl"]
questions = read_jsonl(file_paths)

In [5]:
with jsonlines.open("/<YOUR_OWN_PATH>/ToolQA/data/questions/easy/scirex-easy.jsonl", mode="w") as writer:
    question_indices = random.sample(range(len(questions)), 100)
    question_id = 0
    for id in question_indices:
        row = questions[id]
        row["qid"] = "easy-scirex-{:0>4d}".format(question_id)
        question_id += 1
        writer.write(row)